#  Scipyによる最適化 -第4回-

Scientific computing tools or Python

https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

SciPyは，Pythonによる科学技術計算のためのオープンソースソフトウェアを利用可能な仕組みである。

Scipyは，NumPyパッケージを内部で用いている。具体的には，NumPyで定義された基本的なデータ構造（配列や行列）や，基本的な演算を用いている。また，図示するためにはMatplotlibを用いている。




# 最小二乗法

SciPyの機能を用いると，上下限制約をもつ非線形な最小二乗法を解くことができる。

$$
\begin{array}{ll}
\text{最小化}_x & \frac{1}{2} \sum_{i=1}^m  \rho\left( f_i\left( x \right)^2  \right) \\ 
\text{制約条件} & lb \leq  x \leq ub 
\end{array}
$$

ここで，$f_i(x)$は$n$次元実数空間$\mathbb{R}^n$から$\mathbb{R}$ への滑らかな関数とする。この関数のことを，残差ということにする。また，$\rho(\cdot)$はスカラー値関数とする。この関数は，損失関数と呼ばれる。損失関数は，ロバスト差にかかわるものである。損失関数が線形であれば，通常の最小二乗法となる。

ここでは，次式で定められる式を用いる。

$$
f_i(x) = \frac{x_0 \left(u_i^2+u_ix_1\right)}{u_i^2+u_ix_2+x_3} - y_i,\quad i=0,1,...,10
$$

これは，回帰モデルを表す。$y_i$は観測値を表し，$u_i$は独立変数を表す。未知のパラメータベクトルは，$x=\left(x_0,x_1,x_2,x_3\right)^{\top}$である。

この関数$f(x)$のヤコビ行列は，次のようになる。

$$
J_{i0} = \frac{\partial f_i}{\partial x_0}=\frac{u_i^2+u_ix_1}{u_i^2+u_ix_2+x_3}
$$
$$
J_{i1}=\frac{\partial f_i}{\partial x_1}=\frac{u_i x_0}{u_i^2+u_ix_2+x_3}
$$
$$
J_{i2}=\frac{\partial f_i}{\partial x_2}=-\frac{x_0\left(u_i^2+u_ix_1\right)u_i}{\left(u_i^2+u_ix_2+x_3\right)^2}
$$
$$
J_{i3}=\frac{\partial f_i}{\partial x_3}=-\frac{x_0\left(u_i^2+u_ix_1\right)}{\left(u_i^2+u_ix_2+x_3\right)^2}
$$

最小二乗法によってパラメータ$x$を求めるプログラムは，次のとおりである。

In [ ]:
from scipy.optimize import least_squares

In [ ]:
def model(x, u):
    return x[0] * (u ** 2 + x[1] * u) / (u ** 2 + x[2] * u + x[3])

In [ ]:
def fun(x, u, y):
    return model(x, u) - y

In [ ]:
#ヤコビ行列の定義　
def jac(x, u, y):
    J = np.empty((u.size, x.size))
    den = u ** 2 + x[2] * u + x[3]
    num = u ** 2 + x[1] * u
    J[:, 0] = num / den
    J[:, 1] = x[0] * u / den
    J[:, 2] = -x[0] * num * u / den ** 2
    J[:, 3] = -x[0] * num / den ** 2
    return J

In [ ]:
#データuとyの設定
u = np.array([4.0, 2.0, 1.0, 5.0e-1, 2.5e-1, 1.67e-1, 1.25e-1, 1.0e-1,
              8.33e-2, 7.14e-2, 6.25e-2])
y = np.array([1.957e-1, 1.947e-1, 1.735e-1, 1.6e-1, 8.44e-2, 6.27e-2,
              4.56e-2, 3.42e-2, 3.23e-2, 2.35e-2, 2.46e-2])
x0 = np.array([2.5, 3.9, 4.15, 3.9])

In [ ]:
#最小二乗法による最適解の計算
res = least_squares(fun, x0, jac=jac, bounds=(0, 100), args=(u, y), verbose=1)
#結果を表示する
res.x

得られた結果を図示するには，次のプログラムを実行する。

In [ ]:
import matplotlib.pyplot as plt
u_test = np.linspace(0, 5)
y_test = model(res.x, u_test)
plt.plot(u, y, 'o', markersize=4, label='data')
plt.plot(u_test, y_test, label='fitted model')
plt.xlabel("u")
plt.ylabel("y")
plt.legend(loc='lower right')
plt.show()

--------

## 課題5

(1) `least_squares()`の引数として`loss=soft_l1`を指定すると，何が計算されるかを説明せよ。その際，適切に数式を用いることが必要である．

(2) 上記のデータ`u`と`y`に対して，`loss=soft_l1`を指定して`least_squares()`を実行せよ。そして，得られた結果を，`loss=soft_l1`を指定していない場合の結果と比較せよ．  

(3) データ`u`と`y`の値を（上記の例とは異なるものに）適当に設定し，最小二乗法により回帰モデルを求めよ。また，その結果について説明せよ。

--------




In [ ]:
#ここにプログラムを作成してください．

#最小二乗法による最適解の計算
#res = ...
#結果を表示する
res.x

In [ ]:
#ここにプログラムを作成してください．

import matplotlib.pyplot as plt
u_test = np.linspace(0, 5)
#y_test = ...
#plt.plot(...)
#plt.plot(...)
#... 横軸と縦軸の設定，凡例の表示など
#...
plt.show()

# 方程式の求解

SciPyでは，`root`関数を用いて方程式の解を求めることができる。

## スカラー関数の場合

単一変数のスカラー関数は，いくつかの方法で解を求めることができる。

In [ ]:
import numpy as np 
from scipy.optimize import root 
def func(x):
    return x + 2*np.cos(x)
sol=root(func,0.3)
sol.x

この結果から，この方程式の解は，-1.03であることがわかる。

--------

## 課題6


次の方程式の解を，`root`関数を用いて求めよ。

$$
0.9x^4-4x^3+0.5x^2+11x-4.5=0
$$

--------


In [ ]:
#ここにプログラムを作成してください．

import numpy as np 
from scipy.optimize import root 
#def func_ex6(x):
#    return ...
#def func1(x):
#    return ...
#sol=root(...)
sol.x

## 不動点を求めること

方程式の解を求めることと近い処理は，関数の不動点を求める処理である。関数$g(x)$の不動点は，$g(x)=x$を満たす点$x$のことを指す。ここで，$f(x)$を$f(x)=g(x)-x$と定義すると，$g(x)$の不動点は，方程式$f(x)=0$の解である。SciPyの関数`fixed_point`は，反復法により関数$g$の不動点を得るルーチンである。

## 複数の方程式の解

複数の非線形方程式の解を求めるには，`root`関数を用いる。次の方程式
$$
x+2\cos(x)=0 
$$
の解を求めるためのプログラムは次のとおりである。

まず，解を求める方程式を定める数式を返す関数`func(x)`として定義する。そして，引数として関数`func`と反復の初期点0.3を与えて`root`を実行する。

次に，複数の非線形方程式を満たす点を求める。
$$
\begin{array}{l}
x_0\cos(x_1)=4  \\
x_0x_1 - x_1 = 5
\end{array}
$$
この関数を満たす点は，
$$
f(x)=
\begin{bmatrix}
c_0(x) \\
c_1(x)
\end{bmatrix}
=
\begin{bmatrix}
x_0 \cos(x_1)-4 \\
x_0x_1-x_1-5
\end{bmatrix}
=
\begin{bmatrix}
0 \\ 0
\end{bmatrix}
$$
を満たす点である。

この関数$f(x)$のヤコビ行列$J(x)$は，次のとおりである。
$$
J(x)=\begin{bmatrix}
\frac{\partial c_0}{\partial x_0} & \frac{\partial c_0}{\partial x_1} \\ 
\frac{\partial c_1}{\partial x_0} & \frac{\partial c_1}{\partial x_1}
\end{bmatrix}
=
\begin{bmatrix}
\cos(x_1) & -x_0 \sin(x_1) \\ 
x_1 & x_0-1
\end{bmatrix}
$$

この方程式系の解を`root`関数を求めるために，関数`func2()`を定義する。この関数は，2つの配列`f`と`df`を返す。`f`は解を求める2つの方程式自体であり，`df`はヤコビ行列である．　

In [ ]:
def func2(x):
    f = [x[0] * np.cos(x[1]) - 4,
         x[1]*x[0] - x[1] - 5]
    df = np.array([[np.cos(x[1]), -x[0] * np.sin(x[1])],
                   [x[1], x[0] - 1]])
    return f, df


こうして定義した関数`func2`を，`root`の第1引数として与える。第2引数には初期点`[1,1]`を与える.また，`jac=True`とすることで，第1引数として指定した関数`func2`は，ヤコビ行列も第2の返り値として返すことを関数に教える。`method='lm'`は，アルゴリズムとしてLevenberg-Marquardt法を用いるように指示する。

In [ ]:
sol = root(func2, [1, 1], jac=True, method='lm')
sol.x

これより，解は，$(x_0,x_1)=(6.50,0.91)$であることがわかる。

--------

## 課題7

次の方程式系の解を，`root`を用いて求めよ。その際，ヤコビ行列を用いること。

$$
\begin{array}{l}
x_0+2x_1-2=0\\
x_0^2+4x_1^2-4=0
\end{array}
$$


--------

In [ ]:
#ここにプログラムを作成してください．

from numpy import array
from scipy.optimize import root

#def function(x):
#    f = array( )
#    df = array( )
#    return ...

#sol = root()
print(sol)

# 最適化技術実験 第4回　レポート
### 学生番号 000000 氏名 青学太郎

(ここから下に，レポートを作成してください。レポートを作成したら，「File」->「Save and Checkpoint」でこのjupyter notebookファイルを保存し，さらに，「File」->「Download as」->「Notebook(.ipynb)」を選択して手元のPCにダウンロードしてください．ダウンロードしたファイルの名前を「学生番号-氏名.ipynb」に変更して，CoursePowerから提出してください．ここで，ファイル名内の"学生番号"は1から始まる自分の学生番号に，"氏名"は自分の氏名に置き換えてください．）